数据集地址：https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv

# 文本分类实列

## 1. 导入相关包

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification  

## 2. 加载数据

In [24]:
import pandas as pd
data = pd.read_csv("ChnSentiCorp_htl_all.csv")
# 删除无用的行
data.dropna()

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## 3. 创建Dataset-一次返回一条

In [25]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        # 在pandas中iloc允许你通过行和列的整数位置来访问 DataFrame 中的元素。
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)


In [26]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## 4.划分数据集

In [27]:
from torch.utils.data import random_split

# lengths=[0.9, 0.1] 训练集0.9和验证集0.1，当使用比例拆分时，拆分的值和要为1
# random_split： 数据集划分
traninset, vaildset = random_split(dataset=dataset, lengths=[0.9, 0.1])
# len(traninset), len(vaildset)

## 5. 创建Dataloader-一次返回多条


In [28]:
import torch
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
        
    # 在训练过程中，许多Transformers模型（如BERT、RoBERTa等）的训练循环（training loop）期望输入数据和标签以字典的形式提供。
    # 通常，输入数据包含 input_ids、attention_mask 等键，而标签则包含在 labels 键下。
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs


In [29]:
# DataLoader 是一个用于批量加载数据的实用工具。它可以从数据集（如 Dataset 对象）中提取数据，并将它们组合成批次（batches）。
from torch.utils.data import DataLoader  

# batch_size： 一次取的多少数据；
# shuffle：1 打乱顺序
# collate_fn：用于自定义如何将单个样本组合成一个批次。
traniloader = DataLoader(traninset, batch_size=32, shuffle=True, collate_fn=collate_func)
vaildloader = DataLoader(vaildset, batch_size=64, shuffle=False, collate_fn=collate_func)
next(enumerate(vaildloader))

(0,
 {'input_ids': tensor([[ 101, 2145, 6225,  ..., 3198, 6206,  102],
         [ 101, 6983, 2421,  ..., 7579, 1825,  102],
         [ 101, 6821,  702,  ...,  720, 6814,  102],
         ...,
         [ 101,  769, 6858,  ...,    0,    0,    0],
         [ 101, 4384, 1862,  ...,    0,    0,    0],
         [ 101, 2899,  758,  ..., 6821, 3416,  102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 1, 1, 1],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
         0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 6 创建模型及优化器

In [30]:
# 导入优化器
from torch.optim import Adam
# 创建模型
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("rbt3")

if torch.cuda.is_available():
    # 将模型放到GPU运行
    model = model.cuda()

# model.parameters(): 这是一个生成器，返回模型中所有需要优化的参数。这些参数将在训练过程中通过反向传播进行更新。
# lr=2e-5: 学习率
optimizer = Adam(model.parameters(), lr=2e-5)


OSError: rbt3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

## 7.训练与验证

In [32]:
def evaluate():
    # model.eval()：将模型设置为评估模式，关闭 dropout 和 batch normalization 等训练时使用的特殊层。
    model.eval()
    acc_num = 0
    # 在推理模式下运行，禁用梯度计算，以节省内存和提高速度。
    with torch.inference_mode():
        for batch in vaildloader:
            if torch.cuda.is_available():
                # 用于将字典 batch 中的所有值（通常是张量）移动到 GPU 上。
                # v.cuda()v（通常是一个张量）移动到 GPU 上
                # batch = {
                    # 'input_ids': torch.tensor([1, 2, 3]),
                    # 'attention_mask': torch.tensor([1, 1, 1])
                    # }
                # batch = {k: v.cuda() for k, v in batch.items()} 之后变为 ·----
                # batch = {
                        # 'input_ids': torch.tensor([1, 2, 3]).cuda(),
                        # 'attention_mask': torch.tensor([1, 1, 1]).cuda()
                    # }
                batch = {k: v.cuda() for k, v in batch.items()}
            # 前向传播
            output = model(**batch)
            # 这行代码的作用是从模型的输出中提取预测标签
            # demo: 
            """import torch

               # 假设 output.logits 是一个包含 logits 的张量
               output = {"logits": torch.tensor([
                     [2.5, 1.3, 0.4], # 样本1
                     [0.1, 3.2, 1.5], # 样本2
                     [1.2, 0.8, 2.1], # ...
                     [0.5, 1.7, 2.9]
                 ])}

              # 提取预测标签
              pred = torch.argmax(output["logits"], dim=-1)

              print(pred)  # 输出: tensor([0, 1, 2, 2])"""
            pred = torch.argmax(output.logits, dim=-1)
            print("pred....", pred)
            # pred.long() 张量转换为长整型（long）
            # .float()：将布尔型张量转换为浮点型张量，其中 True 变为 1.0，False 变为 0.0
            
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(vaildset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        # model.train：将模型设置为训练模式。
        model.train()
        for batch in traniloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            # 清零梯度。
            optimizer.zero_grad()
            # 模型进行前向传播
            output = model(**batch)
            # 反向传播计算梯度
            output.loss.backward()
            # 更新模型参数
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        # 调用 evaluate 函数计算并打印准确率。
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

In [22]:
train()

NameError: name 'train' is not defined

In [31]:
sen = "周丹奇觉得这家酒店不好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    print("inputs--",inputs)
    logits = model(**inputs).logits
    # torch.argmax 是 PyTorch 中的一个函数，用于返回输入张量中指定维度上的最大值的索引。
    # 假设你有一个形状为 (batch_size, num_classes) 的张量 logits，表示模型的输出概率分布。你可以使用 torch.argmax 来获取每个样本的预测类别。
    # predicted_classes = torch.argmax(logits, dim=-1)
    # print(predicted_classes) -----输出：tensor([1, 1, 2])
    print("logits--", logits)
    pred = torch.argmax(logits, dim=-1)
    print("pred--",pred)
    print("pred.item",pred.item())
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

NameError: name 'model' is not defined

In [23]:
from transformers import pipeline
model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)
pipe(sen)

[{'label': '差评！', 'score': 0.8683540225028992}]